## SimpleImputer
### This notebook outlines the usage of Simple Imputer (Univariate Imputation).
### Simple Imputer substitutes missing values statistics (mean, median, ...)
#### Dataset: [https://github.com/subashgandyer/datasets/blob/main/heart_disease.csv]

**Demographic**
- Sex: male or female(Nominal)
- Age: Age of the patient;(Continuous - Although the recorded ages have been truncated to whole numbers, the concept of age is continuous)

**Behavioral**
- Current Smoker: whether or not the patient is a current smoker (Nominal)
- Cigs Per Day: the number of cigarettes that the person smoked on average in one day.(can be considered continuous as one can have any number of cigarettes, even half a cigarette.)

**Medical(history)**
- BP Meds: whether or not the patient was on blood pressure medication (Nominal)
- Prevalent Stroke: whether or not the patient had previously had a stroke (Nominal)
- Prevalent Hyp: whether or not the patient was hypertensive (Nominal)
- Diabetes: whether or not the patient had diabetes (Nominal)

**Medical(current)**
- Tot Chol: total cholesterol level (Continuous)
- Sys BP: systolic blood pressure (Continuous)
- Dia BP: diastolic blood pressure (Continuous)
- BMI: Body Mass Index (Continuous)
- Heart Rate: heart rate (Continuous - In medical research, variables such as heart rate though in fact discrete, yet are considered continuous because of large number of possible values.)
- Glucose: glucose level (Continuous)

**Predict variable (desired target)**
- 10 year risk of coronary heart disease CHD (binary: “1”, means “Yes”, “0” means “No”)

In [164]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv("heart_disease.csv")
df

### How many Categorical variables in the dataset?

In [ ]:
df.info()

### How many Missing values in the dataset?
Hint: df.Series.isna( ).sum( )

In [ ]:
for i in range(len(df.columns)):
    missing_data = df[df.columns[i]].isna().sum()
    perc = missing_data / len(df) * 100
    print(f'Feature {i+1} >> Missing entries: {missing_data}  |  Percentage: {round(perc, 2)}')

### Bonus: Visual representation of missing values

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df.isna(), cbar=False, cmap='viridis', yticklabels=False)

### Import SimpleImputer

In [169]:
from sklearn.impute import SimpleImputer

### Create SimpleImputer object with 'mean' strategy

In [170]:
imputer = SimpleImputer(strategy='mean')

### Optional - converting df into numpy array (There is a way to directly impute from dataframe as well)

In [171]:
data = df.values

In [172]:
X = data[:, :-1]
y = data[:, -1]

### Fit the imputer model on dataset to calculate statistic for each column

In [ ]:
imputer.fit(X)

### Trained imputer model is applied to dataset to create a copy of dataset with all filled missing values from the calculated statistic using transform( ) 

In [174]:
X_transform = imputer.transform(X)

### Sanity Check: Whether missing values are filled or not

In [ ]:
# Check missing values in original data
print("Missing values before imputation:")
print(f"Total missing values: {df.isna().sum().sum()}")

In [ ]:
# Check missing values in transformed data
print("Missing values after imputation:")
print(f"Total missing values: {df_transform.isna().sum().sum()}")

### Let's try to visualize the missing values.

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df.isna(), cbar=False, cmap='viridis', yticklabels=False)

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(X_transform.isna(), cbar=False, cmap='viridis', yticklabels=False)

### What's the issue here?
#### Hint: Heatmap needs a DataFrame and not a Numpy Array

In [ ]:
# Convert transformed array back to DataFrame for visualization
df_transform = pd.DataFrame(X_transform, columns=df.columns[:-1])
df_transform

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df_transform.isna(), cbar=False, cmap='viridis', yticklabels=False)

# Check if these datasets contain missing data
### Load the datasets

In [181]:
X_train = pd.read_csv("X_train.csv")
Y_train = pd.read_csv("Y_train.csv")
Y_test = pd.read_csv("Y_test.csv")
X_test = pd.read_csv("X_test.csv")

In [ ]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(X_train.isna(), cbar=False, cmap='viridis', yticklabels=False)

### Is there missing data in this dataset???

In [ ]:
# Check missing values in all datasets
print("Missing values in X_train:", X_train.isna().sum().sum())
print("Missing values in Y_train:", Y_train.isna().sum().sum())
print("Missing values in X_test:", X_test.isna().sum().sum())
print("Missing values in Y_test:", Y_test.isna().sum().sum())

# Build a Logistic Regression model Without imputation

In [185]:
df=pd.read_csv("heart_disease.csv")
X = df[df.columns[:-1]]
y = df[df.columns[-1]]

In [186]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [187]:
model = LogisticRegression()

In [ ]:
model.fit(X,y)

# Drop all rows with missing entries - Build a Logistic Regression model and benchmark the accuracy

In [189]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score

In [ ]:
df=pd.read_csv("heart_disease.csv")
df

In [ ]:
df.shape

### Drop rows with missing values

In [192]:
# Drop rows with missing values
df_clean = df.dropna()

### Split dataset into X and y

In [ ]:
X = df_clean[df_clean.columns[:-1]]
X.shape

In [ ]:
y = df_clean[df_clean.columns[-1]]
y.shape

### Create a pipeline with model parameter

In [195]:
# Create pipeline with LogisticRegression
pipeline = Pipeline([
    ('model', LogisticRegression())
])

### Create a RepeatedStratifiedKFold with 10 splits and 3 repeats and random_state=1

In [196]:
# Create cross-validation object
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

### Call cross_val_score with pipeline, X, y, accuracy metric and cv

In [223]:
# Evaluate model using cross-validation
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

In [ ]:
scores

### Print the Mean Accuracy and Standard Deviation from scores

In [ ]:
print(f"Mean Accuracy: {round(np.mean(scores), 3)}  | Std: {round(np.std(scores), 3)}")

# Build a Logistic Regression model with SimpleImputer Mean Strategy

In [226]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score

In [ ]:
df=pd.read_csv("heart_disease.csv")
df

### Split dataset into X and y

In [ ]:
df.shape

In [ ]:
X = df[df.columns[:-1]]
X.shape

In [ ]:
y = df[df.columns[-1]]
y.shape

### Create a SimpleImputer with mean strategy

In [231]:
# Create SimpleImputer with mean strategy
imputer = SimpleImputer(strategy='mean')

### Create a Logistic Regression model

In [232]:
# Create LogisticRegression model
model = LogisticRegression()

### Create a pipeline with impute and model parameters

In [233]:
# Create pipeline with imputer and model
pipeline = Pipeline([
    ('imputer', imputer),
    ('model', model)
])

### Create a RepeatedStratifiedKFold with 10 splits and 3 repeats and random_state=1

In [234]:
# Create cross-validation object
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

### Call cross_val_score with pipeline, X, y, accuracy metric and cv

In [247]:
# Evaluate model using cross-validation
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

In [ ]:
scores

### Print the Mean Accuracy and Standard Deviation

In [ ]:
print(f"Mean Accuracy: {round(np.mean(scores), 3)}  | Std: {round(np.std(scores), 3)}")

### Which accuracy is better? 
- Dropping missing values
- SimpleImputer with Mean Strategy

In [ ]:
print("Comparison of Strategies:")
print("-" * 50)
print("1. Dropping missing values:")
print(f"Mean Accuracy: {round(0.848, 3)}  | Std: {round(0.036, 3)}")
print("\n2. SimpleImputer with Mean Strategy:")
print(f"Mean Accuracy: {round(0.851, 3)}  | Std: {round(0.035, 3)}")
print("\nConclusion: SimpleImputer with Mean Strategy performs slightly better")
print("and preserves more data compared to dropping rows with missing values.")

# SimpleImputer Mean - Benchmark after Mean imputation with RandomForest

### Import libraries

In [251]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score

### Create a SimpleImputer with mean strategy

In [252]:
# Create SimpleImputer with mean strategy
imputer = SimpleImputer(strategy='mean')

### Create a RandomForest model

In [253]:
# Create RandomForest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=1)

### Create a pipeline

In [254]:
# Create pipeline with imputer and RandomForest
pipeline = Pipeline([
    ('imputer', imputer),
    ('model', rf_model)
])

### Create RepeatedStratifiedKFold

In [255]:
# Create cross-validation object
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

### Create Cross_val_score

In [256]:
# Evaluate model using cross-validation
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

### Print Mean Accuracy and Standard Deviation

In [ ]:
# Print results
print(f"Mean Accuracy: {round(np.mean(scores), 3)}  | Std: {round(np.std(scores), 3)}")

# Assignment
# Run experiments with different Strategies and different algorithms

## STRATEGIES
- Mean
- Median
- Most_frequent
- Constant

## ALGORITHMS
- Logistic Regression
- KNN
- Random Forest
- SVM
- Any other algorithm of your choice

#### Hint: Collect the pipeline creation, KFold, and Cross_Val_Score inside a for loop and iterate over different strategies in a list and different algorithms in a list

In [278]:
# Import required models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

# Define strategies and models to test
strategies = ['mean', 'median', 'most_frequent', 'constant']
models = [
    ('Logistic Regression', LogisticRegression(max_iter=10000)), # Increase max_iter to avoid convergence warnings
    ('KNN', KNeighborsClassifier()),
    ('Random Forest', RandomForestClassifier(n_estimators=100, random_state=1)),
    ('SVM', SVC(random_state=1)),
    ('Gradient Boosting', GradientBoostingClassifier(n_estimators=100, random_state=1))
]

# Store results
results = []

# Create cross-validation object
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# Iterate through all combinations
for strategy in strategies:
    for model_name, model in models:
        # Create pipeline
        pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy=strategy)),
            ('model', model)
        ])
        
        # Evaluate model
        scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
        
        # Store results
        results.append({
            'strategy': strategy,
            'model': model_name,
            'mean_accuracy': np.mean(scores),
            'std': np.std(scores)
        })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Display results in a formatted table
print("Results of all combinations:")
print("-" * 80)
for _, row in results_df.iterrows():
    print(f"Strategy: {row['strategy']:<15} Model: {row['model']:<20} "
          f"Accuracy: {row['mean_accuracy']:.3f} ± {row['std']:.3f}")

Results of all combinations:
--------------------------------------------------------------------------------
Strategy: mean            Model: Logistic Regression  Accuracy: 0.855 ± 0.005
Strategy: mean            Model: KNN                  Accuracy: 0.837 ± 0.009
Strategy: mean            Model: Random Forest        Accuracy: 0.850 ± 0.006
Strategy: mean            Model: SVM                  Accuracy: 0.848 ± 0.002
Strategy: mean            Model: Gradient Boosting    Accuracy: 0.846 ± 0.008
Strategy: median          Model: Logistic Regression  Accuracy: 0.855 ± 0.006
Strategy: median          Model: KNN                  Accuracy: 0.836 ± 0.008
Strategy: median          Model: Random Forest        Accuracy: 0.850 ± 0.006
Strategy: median          Model: SVM                  Accuracy: 0.848 ± 0.002
Strategy: median          Model: Gradient Boosting    Accuracy: 0.846 ± 0.009
Strategy: most_frequent   Model: Logistic Regression  Accuracy: 0.855 ± 0.006
Strategy: most_frequent   Model:

# Q1: Which is the best strategy for this dataset using Random Forest algorithm?
- MEAN
- MEDIAN
- MOST_FREQUENT
- CONSTANT

In [279]:
# Filter results for Random Forest
rf_results = results_df[results_df['model'] == 'Random Forest']
print("Random Forest performance with different strategies:")
print("-" * 50)
for _, row in rf_results.iterrows():
    print(f"Strategy: {row['strategy']:<15} Accuracy: {row['mean_accuracy']:.3f} ± {row['std']:.3f}")
    
best_rf = rf_results.loc[rf_results['mean_accuracy'].idxmax()]
print(f"\nBest strategy for Random Forest: {best_rf['strategy']} with accuracy {best_rf['mean_accuracy']:.3f} ± {best_rf['std']:.3f}")

Random Forest performance with different strategies:
--------------------------------------------------
Strategy: mean            Accuracy: 0.850 ± 0.006
Strategy: median          Accuracy: 0.850 ± 0.006
Strategy: most_frequent   Accuracy: 0.849 ± 0.006
Strategy: constant        Accuracy: 0.848 ± 0.006

Best strategy for Random Forest: mean with accuracy 0.850 ± 0.006


# Q2:  Which is the best algorithm for this dataset using Mean Strategy?
- Logistic Regression
- Random Forest
- KNN
- any other algorithm of your choice (BONUS)

In [280]:
# Filter results for Mean strategy
mean_results = results_df[results_df['strategy'] == 'mean']
print("Mean strategy performance with different algorithms:")
print("-" * 50)
for _, row in mean_results.iterrows():
    print(f"Algorithm: {row['model']:<20} Accuracy: {row['mean_accuracy']:.3f} ± {row['std']:.3f}")
    
best_mean = mean_results.loc[mean_results['mean_accuracy'].idxmax()]
print(f"\nBest algorithm with Mean strategy: {best_mean['model']} with accuracy {best_mean['mean_accuracy']:.3f} ± {best_mean['std']:.3f}")

Mean strategy performance with different algorithms:
--------------------------------------------------
Algorithm: Logistic Regression  Accuracy: 0.855 ± 0.005
Algorithm: KNN                  Accuracy: 0.837 ± 0.009
Algorithm: Random Forest        Accuracy: 0.850 ± 0.006
Algorithm: SVM                  Accuracy: 0.848 ± 0.002
Algorithm: Gradient Boosting    Accuracy: 0.846 ± 0.008

Best algorithm with Mean strategy: Logistic Regression with accuracy 0.855 ± 0.005


# Q3: Which is the best combination of algorithm and best Imputation Strategy overall?
- Mean , Median, Most_frequent, Constant
- Logistic Regression, Random Forest, KNN

In [281]:
# Find best overall combination
best_overall = results_df.loc[results_df['mean_accuracy'].idxmax()]
print("Best overall combination:")
print("-" * 50)
print(f"Strategy: {best_overall['strategy']}")
print(f"Algorithm: {best_overall['model']}")
print(f"Accuracy: {best_overall['mean_accuracy']:.3f} ± {best_overall['std']:.3f}")

Best overall combination:
--------------------------------------------------
Strategy: most_frequent
Algorithm: Logistic Regression
Accuracy: 0.855 ± 0.006


# Analysis Summary

After comparing different imputation strategies and machine learning algorithms, we can draw several conclusions:

1. **Best Imputation Strategy for Random Forest:**
   - Mean and Median strategies tied with accuracy of 0.850 ± 0.006
   - Most_frequent (0.849 ± 0.006) and Constant (0.848 ± 0.006) performed slightly worse
   - The small difference between strategies suggests Random Forest is robust to the choice of imputation method

2. **Best Algorithm with Mean Strategy:**
   - Logistic Regression performed best (0.855 ± 0.005)
   - Random Forest was second (0.850 ± 0.006)
   - KNN performed worst (0.837 ± 0.009)
   - SVM (0.848 ± 0.002) showed the lowest standard deviation

3. **Best Overall Combination:**
   - Most_frequent imputation + Logistic Regression (0.855 ± 0.006)
   - Mean imputation + Logistic Regression performed equally well (0.855 ± 0.005)
   - The small performance differences suggest the dataset is well-behaved

4. **Key Takeaways:**
   - Simple imputation strategies work well for this dataset
   - Logistic Regression consistently outperformed other algorithms
   - The choice of imputation strategy has minimal impact on performance
   - Low standard deviations across all methods indicate stable and reliable predictions